In [97]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder


data = pd.read_csv('ballons.csv')

## converting categorical value to numerical 
# categories = []
# categories = ['Outlook','Temp','Humidity','Wind','Play'] # weather
categories = ['Color','Size','Act','Age','Inflated'] # ballons
# categories = ['top-left-square','top-middle-square','top-right-square','middle-left-square','middle-middle-square','middle-right-square','bottom-left-square','bottom-middle-square','bottom-right-square','Class']
target = ['Yes', 'No']
 
le = LabelEncoder()
 
for i in range(len(categories)):
    
    label = le.fit_transform(data[categories[i]])

    data.drop(categories[i], axis=1, inplace=True)
    
    data[categories[i]] = label 

In [98]:
import numpy as np
from collections import Counter 

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None) -> None:
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # leaf node 
    
    def if_leaf_node(self):
        return self.value is not None

In [99]:
class MyDecisionTree:
    
    def __init__(self, min_sample_split=2, max_depth=10, n_features=None) -> None:
        
        self.min_sample_split = min_sample_split 
        self.max_depth = max_depth 
        
        self.n_features = n_features
        
        self.root = None
        
    def fit(self, X, y):
                
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        
        self.root = self._build_tree(X, y)
                
    def _build_tree(self, X, y, depth=0):
        
        n_samples, n_features = X.shape     # [rows, columns]
        n_labels = len(np.unique(y))        # class variables
        
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            
            if len(y) > 0:
                leaf_node_value = self.calculate_leaf_value(y) 
                return Node(value=leaf_node_value)
            else: return Node(value=0)
        
        feature_indices = np.random.choice(n_features, self.n_features, replace=False)
                
        best_feature, best_threshold = self._find_best_split(X, y, feature_indices)
        
        left_indices, right_indices = self._split_node(X[:, best_feature], best_threshold)
        left = self._build_tree(X[left_indices, :], y[left_indices], depth+1)
        right = self._build_tree(X[right_indices, :], y[right_indices], depth+1)
        return Node(best_feature, best_threshold, left, right)
        
    def calculate_leaf_value(self, y):
          
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        
        return value 
    
    def _find_best_split(self, X, y, feature_indices):
        
        best_gain = -1 
        split_index, split_threshold = None, None
        
        for feature_index in feature_indices:
            
            X_column = X[:, feature_index]  
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                
                if gain > best_gain:
                    best_gain = gain 
                    split_index = feature_index
                    split_threshold = threshold
        
        return split_index, split_threshold     
        
    def _information_gain(self, y, X_column, threshold):
        
        parent_entropy = self._entropy(y)
        
        left_indices, right_indices = self._split_node(X_column, threshold)
        
        if len(left_indices) == 0 or len(right_indices) == 0:
            return 0
        
        total_samples = len(y)
        left_subtree_samples, right_subtree_samples = len(left_indices), len(right_indices)
        entropy_left_subtree, entropy_right_subtree = self._entropy(y[left_indices]), self._entropy(y[right_indices])
        child_entropy = (left_subtree_samples/total_samples) * entropy_left_subtree + (right_subtree_samples/total_samples) * entropy_right_subtree
        
        information_gain = parent_entropy - child_entropy
        return information_gain
        
    def _entropy(self, y):
        
        hist = np.bincount(y) # count the number of each class variable in the array
        ps = hist/len(y)      # probability of each class variable (#p/n)
        
        return -np.sum([p * np.log(p) for p in ps if p>0])
        
    def _split_node(self, X_column, threshold):
        
        
        left_indices = np.argwhere(X_column <= threshold).flatten()
        right_indices = np.argwhere(X_column > threshold).flatten()
        return left_indices, right_indices
            
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
            
    def _traverse_tree(self, x, node):
        
        if node.if_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:  
            return self._traverse_tree(x, node.left)
        
        else:
            return self._traverse_tree(x, node.right)
        
    def print_tree(self, root):
        self._print_node(root)
        
    def _print_node(self, node, indent=""):
        if node is None:
            return
        if node.if_leaf_node():
            print(indent + "Leaf Node: " + str(node.value))
        else:
            print(indent + "Feature [LEFT] " + categories[(node.feature)] + " <= " + target[node.threshold] + "?")
            self._print_node(node.left, indent + "|__")
            print(indent + "Feature [RIGHT] " + categories[(node.feature)] )
            self._print_node(node.right, indent + "|__")
               

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score


class RandomForest:
    
    def __init__(self, n_estimators=11, max_depth=10, n_features=3) -> None:
        
        # number of decision trees
        self.n_estimators = n_estimators
        
        self.max_depth = max_depth
        self.n_features = n_features
        self.tree = []
        
    def fit(self, X, y):
        
        for _ in range(self.n_estimators):
            
            decision_tree = MyDecisionTree(n_features=self.n_features)
            
            sample_indices = np.random.choice(len(X), len(X), replace=True)
            X_sample = X[sample_indices]
            y_sample = y[sample_indices]
            
            decision_tree.fit(X_sample, y_sample)
            self.tree.append(decision_tree)
    
    def predict(self, X):
        
        # stack the lists verticallyy
        arr = np.vstack([tree.predict(X) for tree in self.tree])

        # define function to find the most frequent value 
        def most_frequent_value(arr):
            return np.argmax(np.bincount(arr))

        result = np.apply_along_axis(most_frequent_value, axis=0, arr=arr)
        return result
                

In [101]:
# predictiom using random forest     
train, test = train_test_split(data, test_size=.5)

X_test, Y_test = test.iloc[:,:-1].values, test.iloc[:,-1].values
X_train, Y_train = train.iloc[:,:-1].values, train.iloc[:,-1].values

random_forest = RandomForest()
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
# print(Y_pred)

# print the confusion matrix and accuracy
print("\nConfusion Matrix: \n", confusion_matrix(Y_test, Y_pred))
print("Accuracy: ",accuracy_score(Y_test, Y_pred)*100, "%")  


Confusion Matrix: 
 [[6 2]
 [0 2]]
Accuracy:  80.0 %


In [102]:
# Decision Tree from sklearn
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5, n_estimators=100)

clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
print(Y_pred)

print("Confusion Matrix: \n", confusion_matrix(Y_test, Y_pred))
print("Accuracy: ",accuracy_score(Y_test, Y_pred)*100, "%")

[1 0 0 1 0 1 0 0 0 1]
Confusion Matrix: 
 [[6 2]
 [0 2]]
Accuracy:  80.0 %
